#Pertemuan 3

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#import apriori from library mixtend
from mlxtend.frequent_patterns import apriori,association_rules
import warnings
warnings.filterwarnings("ignore")

##Dataset
* Membuat dataset transaksi sebagai dataframe
* Cek dataset transaksi

In [ ]:
#membuat data tabular yang terdiri dari data transaksi
df = pd.DataFrame(
    {
        "ID Transaksi": pd.Series([1,2,3,4,5]),
        "Item": pd.Series([["Susu,Roti","Biskuit","Mentega"],\
                           ["Susu","Biskuit","Gula","Kopi"],\
                           ["Gula","Kopi","Susu"], ["Kopi","Biskuit","Susu","Gula"],\
                           ["Mentega","Roti","Kopi"]]),
    }
)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID Transaksi  5 non-null      int64 
 1   Item          5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


##Encoding data Transaksi

In [ ]:
df = df.explode('Item')
df

,ID Transaksi,Item
0,1,"Susu,Roti"
0,1,Biskuit
0,1,Mentega
1,2,Susu
1,2,Biskuit
1,2,Gula
1,2,Kopi
2,3,Gula
2,3,Kopi
2,3,Susu


In [ ]:
basket = (df.groupby(['ID Transaksi','Item'])['Item'].count()\
          .unstack().reset_index().fillna(0)\
          .set_index('ID Transaksi'))
basket.head()

Item,Biskuit,Gula,Kopi,Mentega,Roti,Susu,"Susu,Roti"
ID Transaksi,,,,,,,
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,1.0,1.0,1.0,0.0,0.0,1.0,0.0
3,0.0,1.0,1.0,0.0,0.0,1.0,0.0
4,1.0,1.0,1.0,0.0,0.0,1.0,0.0
5,0.0,0.0,1.0,1.0,1.0,0.0,0.0


##Algoritma Apriori
* Algoritma apriori ditujukaqn untuk mengidentifikasi item-item yang sering muncul bersama dalam suatu transaksi atau kumpulan data.
* Algoritma Apriori sangat umum digunakan dalam analisis keranjang belanja, sistem rekomendasi, dan pemahaman pola konsumen
* Eksekusi algoritma apriori menggunakan library mlextend dengan fungsi apriori
* Langkah-langkah umum apriori : mengeliminasi itemset berdasarkan minimum support (threshold), menghitung confidence sejauh perulangan tertentu, menemukan cross product atau kombinasi item-item yang muncul dalam suatu transaksi

In [ ]:
#Total per item
df_encode = df.groupby("Item")["Item"].count().reset_index(name="Count")
df_encode

,Item,Count
0,Biskuit,3
1,Gula,3
2,Kopi,4
3,Mentega,2
4,Roti,1
5,Susu,3
6,"Susu,Roti",1


# Apriori Tahap 1
* Eliminasi itemset berdasarkan minimum support dan lakukan iterasi

##Iterasi I
* Hitung support setiap item
* Asumsikan minimum support 3 atau 60% (muncul 3x dan 5 transaksi)
* Filter item yang tidak memenuhi minimum support

In [ ]:
def create_support(x):
  #jumlah transaksi per item
  N = 5 #jumlah transaksi per item
  y = (x/N)*100 #hitung persentase
  return y

In [ ]:
df_i1 = df_encode.copy() #copy dataset df_encode ke df_i1 (df iterasi 1)
df_i1["Support"] = df_i1["Count"].apply(lambda x: create_support(x)) #apply fungsi ke df_il dengan tambahan kolom baru
df_i1

,Item,Count,Support
0,Biskuit,3,60.0
1,Gula,3,60.0
2,Kopi,4,80.0
3,Mentega,2,40.0
4,Roti,1,20.0
5,Susu,3,60.0
6,"Susu,Roti",1,20.0


In [ ]:
df_i1 = df_i1[df_i1['Support'] >= 60] #filter data yang tidak memenuhi 60%
df_i1

,Item,Count,Support
0,Biskuit,3,60.0
1,Gula,3,60.0
2,Kopi,4,80.0
5,Susu,3,60.0


##Iterasi II
* Kombinasi setiap item menjadi

In [ ]:
# cek supportnya
filtered = ["Roti","Mentega"]
df_filtered = df[~df["Item"].isin(filtered)]
df_encode_2 = df_filtered.groupby("Item")["Item"].count().reset_index(name="Count")
df_i2 = df_encode_2.copy()
df_i2["Support"] = df_i2["Count"].apply(lambda x : create_support(x))
df_i2

,Item,Count,Support
0,Biskuit,3,60.0
1,Gula,3,60.0
2,Kopi,4,80.0
3,Susu,3,60.0
4,"Susu,Roti",1,20.0


In [ ]:
df_i2 = df_i2[df_i2['Support']>=60] #filter data yang tidak memenuhi 60%
df_i2

,Item,Count,Support
0,Biskuit,3,60.0
1,Gula,3,60.0
2,Kopi,4,80.0
3,Susu,3,60.0


##Apriori Tahap 2


##Kombinasi
Cross product atau hasil kombinasi dari item yang sering muncul dalam satu transsaksi

In [ ]:
#cek kombinasi yang dihasilkan
basket_2 = (df_filtered.groupby(['ID Transaksi','Item'])['Item'].count()\
            .unstack().reset_index().fillna(0)\
            .set_index('ID Transaksi'))
frequent_itemsets_2 = apriori(basket_2, min_support=0.6, use_colnames=True)
frequent_itemsets_2
rules = association_rules(frequent_itemsets_2, metric="confidence", min_threshold=0.6)
rules[["antecedents", "consequents", "support", "confidence"]]

,antecedents,consequents,support,confidence
0,(Kopi),(Gula),0.6,0.75
1,(Gula),(Kopi),0.6,1.00
2,(Gula),(Susu),0.6,1.00
3,(Susu),(Gula),0.6,1.00
4,(Kopi),(Susu),0.6,0.75
5,(Susu),(Kopi),0.6,1.00
6,"(Kopi, Gula)",(Susu),0.6,1.00
7,"(Kopi, Susu)",(Gula),0.6,1.00
8,"(Gula, Susu)",(Kopi),0.6,1.00
9,(Kopi),"(Gula, Susu)",0.6,0.75
